In [1]:
# loading transactions and storing in a list of lists

def load_transactions(path):
    Transactions = []
    with open(path, 'r') as df:
        for line in df:
            l = list(line.strip().split(','))
            Transactions.append(l)
    return Transactions

Loading Data

In [2]:
path = 'Datasets/AllElectronics.csv'
Transactions = load_transactions(path)

In [3]:
print(Transactions)

[['I1', 'I2', 'I5'], ['I2', 'I4'], ['I2', 'I3'], ['I1', 'I2', 'I4'], ['I1', 'I3'], ['I2', 'I3'], ['I1', 'I3'], ['I1', 'I2', 'I3', 'I5'], ['I1', 'I2', 'I3']]


Initialising the itemsets

In [4]:
C = {}
L = {}
order = ['I' + str(i) for i in range(1, 6)]
itemsize = 1
InvalidSets = {itemsize: []}
C.update({itemsize: [ [f] for f in order]})
print(C)

{1: [['I1'], ['I2'], ['I3'], ['I4'], ['I5']]}


Create L1

In [5]:
# counts support of the itemset
def countOccurences(itemset, Transactions):
    count = 0
    for i in range(len(Transactions)):
        if set(itemset).issubset(set(Transactions[i])):
            count += 1
    return count

In [6]:
def getFreqItemSet(itemsets, Transactions, minSupport, prevInvalidSubsets):

    sizeInvalid = len(prevInvalidSubsets)
    L = []
    itemSetSupport = []
    newInvalidSets = []
    
    for i in range(len(itemsets)):
        invalid = False
        # checks if any subset of the current itemset does not meet the minimum support threshold
        if sizeInvalid > 0:
            for itemS in prevInvalidSubsets[sizeInvalid]:
                if set(itemS).issubset(set(itemsets[i])):
                    invalid = True
                    break
        # counts support for the itemset
        if not invalid:
            countSup = countOccurences(itemsets[i], Transactions)
            if countSup/len(Transactions) >= minSupport:
                L.append(itemsets[i])
                itemSetSupport.append(countSup)
            else:
                newInvalidSets.append(itemsets[i])
    return L, itemSetSupport, newInvalidSets

In [7]:
def printTable(T, support):
    print("Itemset           | Frequency")
    for i in range(len(T)):
        print("{}   :   {}".format(T[i], support[i]))

In [8]:
supportL = {}
minSupport = 0.2
minConf = 0.7
f, supp, newInvalidSets = getFreqItemSet(C[itemsize], Transactions, minSupport, InvalidSets)
InvalidSets.update({itemsize: newInvalidSets})
L.update({itemsize: f})
supportL.update({itemsize: supp})

In [9]:
printTable(L[1], supportL[1])

Itemset           | Frequency
['I1']   :   6
['I2']   :   7
['I3']   :   6
['I4']   :   2
['I5']   :   2


In [10]:
def joinItemSetsHelper(it1, it2, order):
    it1.sort(key = lambda x: order.index(x))
    it2.sort(key = lambda x: order.index(x))
    
    for i in range(len(it1) - 1):
        # ensures join is happening between two same itemsets
        if it1[i] != it2[i]:
            return []
    # self joining of itemsets
    if order.index(it1[-1]) < order.index(it2[-1]):
        return it1 + [it2[-1]]
        
    return []  

In [11]:
def joinItemSets(setItemSets, order):
    C = []
    for i in range(len(setItemSets)):
        for j in range(i+1, len(setItemSets)):
            it = joinItemSetsHelper(setItemSets[i], setItemSets[j], order)
            if len(it) > 0:
                C.append(it)
    return C

In [12]:
k = itemsize + 1
convergence = False
# updates C for every iteration
while not convergence:
    C.update({ k : joinItemSets(L[k-1], order)})
    print("Table (C{}:  \n)".format(k))
    printTable(C[k], [countOccurences(it, Transactions) for it in C[k]])
    f, supp, newInvalidSets = getFreqItemSet(C[k], Transactions, minSupport, InvalidSets)
    InvalidSets.update({k: newInvalidSets})
    L.update({k: f})
    supportL.update({k : supp})
    if(len(L[k]) == 0):
        convergence = True
    else:
        print("Table (L{}:  \n)".format(k))
        printTable(L[k], supportL[k])
    k += 1

Table (C2:  
)
Itemset           | Frequency
['I1', 'I2']   :   4
['I1', 'I3']   :   4
['I1', 'I4']   :   1
['I1', 'I5']   :   2
['I2', 'I3']   :   4
['I2', 'I4']   :   2
['I2', 'I5']   :   2
['I3', 'I4']   :   0
['I3', 'I5']   :   1
['I4', 'I5']   :   0
Table (L2:  
)
Itemset           | Frequency
['I1', 'I2']   :   4
['I1', 'I3']   :   4
['I1', 'I5']   :   2
['I2', 'I3']   :   4
['I2', 'I4']   :   2
['I2', 'I5']   :   2
Table (C3:  
)
Itemset           | Frequency
['I1', 'I2', 'I3']   :   2
['I1', 'I2', 'I5']   :   2
['I1', 'I3', 'I5']   :   1
['I2', 'I3', 'I4']   :   0
['I2', 'I3', 'I5']   :   1
['I2', 'I4', 'I5']   :   0
Table (L3:  
)
Itemset           | Frequency
['I1', 'I2', 'I3']   :   2
['I1', 'I2', 'I5']   :   2
Table (C4:  
)
Itemset           | Frequency
['I1', 'I2', 'I3', 'I5']   :   1


Generating association rules

In [13]:
from itertools import combinations, chain

def powerset(s):
    return list(chain.from_iterable(combinations(s, r) for r in range(1, len(s) + 1)))

In [23]:
assoRules = ""
for i in range(len(L[3])):
    s = powerset(L[3][i])
    s.pop()
    for j in s:
        S = set(j)
        I = set(L[3][i])
        I_S = set(I-S)
        suppI = countOccurences(I, Transactions)
        conf = suppI/countOccurences(S, Transactions)
        if conf >= minConf and suppI >= minSupport:
            assoRules += "Frequency Itemset: {}\n".format(I)
            assoRules += "Rule: {} => {}\n".format(list(S), list(I_S))
            assoRules += "Confidence: {0:2.3f}\n".format(conf)
            assoRules += "Support:{0:2.3f}\n".format(suppI/len(Transactions))

In [24]:
print(assoRules)

Frequency Itemset: {'I1', 'I2', 'I5'}
Rule: ['I5'] => ['I1', 'I2']
Confidence: 1.000
Support:0.222
Frequency Itemset: {'I1', 'I2', 'I5'}
Rule: ['I1', 'I5'] => ['I2']
Confidence: 1.000
Support:0.222
Frequency Itemset: {'I1', 'I2', 'I5'}
Rule: ['I2', 'I5'] => ['I1']
Confidence: 1.000
Support:0.222

